In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
# import logging
import json
import re

# JSON formatting functions
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    training_data = []
    lines=[]
    with open(dataturks_JSON_FilePath, 'r') as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        text = data['content'].replace("\n", " ")
        entities = []
        data_annotations = data['annotation']
        if data_annotations is not None:
            for annotation in data_annotations:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    point_start = point['start']
                    point_end = point['end']
                    point_text = point['text']

                    lstrip_diff = len(point_text) - len(point_text.lstrip())
                    rstrip_diff = len(point_text) - len(point_text.rstrip())
                    if lstrip_diff != 0:
                        point_start = point_start + lstrip_diff
                    if rstrip_diff != 0:
                        point_end = point_end - rstrip_diff
                    entities.append((point_start, point_end + 1 , label))
        training_data.append((text, {"entities" : entities}))
    return training_data

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data

In [7]:
data = trim_entity_spans(convert_dataturks_to_spacy("/content/Entity Recognition in Resumes.json"))
data[0]

["Abhishek Jha Application Development Associate - Accenture  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a  • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company's growth in best possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK EXPERIENCE  Application Development Associate  Accenture -  November 2017 to Present  Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input. Also, Training the bot for different possible utterances (Both positive and negative), which will be given as input by the user.  EDUCATION  B.E in Information science and engineering  B.v.b college of engineering and technology -  Hubli, Karnataka  August 2013 to June 2017  12th in Mathematics  Woodbine modern school  April 2011 to March 2013  10th  Kendriya Vidyalaya  April 2001 to March 2011  SKILLS  C (Le

In [13]:
df['content'][30]

TypeError: ignored

In [12]:
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
clean=[]

for i in range(0,220):
    review = re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"', ' ', df['content'][i])
    review = review.lower()
    review = review.split()
    lm= WordNetLemmatizer() 
    review = [lm.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    clean.append(review)

NameError: ignored

In [ ]:
df['content'][0]

"Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills\nand knowledge for my individual and company's growth in best possible ways.\n\nWilling to relocate to: Bangalore, Karnataka\n\nWORK EXPERIENCE\n\nApplication Development Associate\n\nAccenture -\n\nNovember 2017 to Present\n\nRole: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries\nfor the Bot which will be triggered based on given input. Also, Training the bot for different possible\nutterances (Both positive and negative), which will be given as\ninput by the user.\n\nEDUCATION\n\nB.E in Information science and engineering\n\nB.v.b college of engineering and technology -  Hubli, Karnataka\n\nAugust 2013 to June 2017\n\n12th in Mathematics\n\nWoodbine modern school\n\nApril 2011 to March 2013\n\n10th\n\nKendriya Vidyalaya\n\

In [ ]:
clean[0]

'abhishek jha application development associate accenture bengaluru karnataka email indeed indeed com r abhishek jha 10e7a8cb732bc43a work organization provides opportunity improve skill knowledge individual company growth best possible way willing relocate bangalore karnataka work experience application development associate accenture november 2017 present role currently working chat bot developing backend oracle peoplesoft query bot triggered based given input also training bot different possible utterance positive negative given input user education b e information science engineering b v b college engineering technology hubli karnataka august 2013 june 2017 12th mathematics woodbine modern school april 2011 march 2013 10th kendriya vidyalaya april 2001 march 2011 skill c le 1 year database le 1 year database management le 1 year database management system le 1 year java le 1 year additional information technical skill programming language c c java oracle peoplesoft internet thing m

In [ ]:
df['cleaned']=clean

In [ ]:
df.head()

,content,annotation,extras,cleaned
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",NaN,abhishek jha application development associate...
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN,afreen jamadar active member iiit committee th...
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",NaN,akhil yadav polemaina hyderabad telangana emai...
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN,alok khandai operational analyst sql dba engin...
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",NaN,ananya chavan lecturer oracle tutorial mumbai ...


In [ ]:
pip install spacy

In [ ]:
import spacy
nlp = spacy.load("en")
from spacy.lang.en import English
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

text = df['cleaned'][100]

my_doc = nlp(text)

In [ ]:
# Create list of word tokens
token_list = []
for token in my_doc:
    token_list.append(token.text)
print(token_list)

['yasothai', 'jayaramachandran', 'lead', 'engineer', 'automation', 'testing', 'chennai', 'tamil', 'nadu', 'email', 'indeed', 'indeed', 'com', 'r', 'yasothai', 'jayaramachandran', 'c36e76b64d9f477f', '4', 'year', 'experience', 'testing', 'methodology', 'automation', 'regression', 'sanity', 'manual', 'hand', 'experience', 'selenium', 'webdriver', 'python', 'automation', 'skilled', 'ide', 'pycharm', 'eclipse', 'pydev', 'plugin', 'selenium', 'testing', 'web', 'browser', 'using', 'python', 'expertise', 'developing', 'automation', 'script', 'testing', 'various', 'component', 'skilled', 'providing', 'effective', 'quality', 'automation', 'process', 'hand', 'experience', 'functional', 'automation', 'script', 'using', 'tcl', 'programming', 'python', 'experienced', 'planning', 'designing', 'developing', 'deploying', 'testing', 'strategy', 'trained', 'worked', 'closely', 'along', 'developer', 'dev', 'test', 'manual', 'engineer', 'automation', 'task', 'good', 'experience', 'reviewing', 'automated',

In [ ]:
import en_core_web_sm

# load en_core_web_sm of English for vocabluary, syntax & entities
nlp = en_core_web_sm.load()

#  "nlp" Objectis used to create documents with linguistic annotations.
docs = nlp(df['cleaned'][100])

for word in docs:
    print(word.text,word.pos_)

yasothai PROPN
jayaramachandran PROPN
lead PROPN
engineer PROPN
automation PROPN
testing PROPN
chennai PROPN
tamil PROPN
nadu PROPN
email NOUN
indeed ADV
indeed ADV
com PROPN
r DET
yasothai PROPN
jayaramachandran PROPN
c36e76b64d9f477f VERB
4 NUM
year NOUN
experience NOUN
testing NOUN
methodology NOUN
automation NOUN
regression NOUN
sanity NOUN
manual ADJ
hand NOUN
experience NOUN
selenium NOUN
webdriver PROPN
python PROPN
automation NOUN
skilled VERB
ide PROPN
pycharm PROPN
eclipse PROPN
pydev PROPN
plugin ADJ
selenium NOUN
testing NOUN
web NOUN
browser NOUN
using VERB
python PROPN
expertise NOUN
developing VERB
automation NOUN
script NOUN
testing NOUN
various ADJ
component NOUN
skilled ADJ
providing VERB
effective ADJ
quality NOUN
automation NOUN
process NOUN
hand NOUN
experience NOUN
functional ADJ
automation NOUN
script NOUN
using VERB
tcl PROPN
programming PROPN
python PROPN
experienced VERB
planning NOUN
designing VERB
developing VERB
deploying VERB
testing NOUN
strategy NOUN
tra

In [ ]:
from spacy import displacy

nytimes= nlp(df['cleaned'][100])

entities=[(i, i.label_, i.label) for i in nytimes.ents]
entities

[(yasothai jayaramachandran, 'PERSON', 380),
 (yasothai jayaramachandran, 'PERSON', 380),
 (4 year, 'DATE', 391),
 (ide pycharm, 'ORG', 383),
 (python, 'ORG', 383),
 (tcl programming python, 'ORG', 383),
 (tcl python, 'ORG', 383),
 (excel csv file, 'ORG', 383),
 (python, 'GPE', 384),
 (scrum, 'PRODUCT', 386),
 (cisco, 'GPE', 384),
 (january 2014 june 2015, 'DATE', 391),
 (jun 2015, 'LAW', 390),
 (20, 'CARDINAL', 397),
 (v6, 'PRODUCT', 386),
 (1000, 'CARDINAL', 397),
 (v6, 'CARDINAL', 397),
 (tcl programming, 'ORG', 383),
 (digital, 'ORG', 383),
 (cisco, 'GPE', 384),
 (qos, 'ORG', 383),
 (cisco, 'GPE', 384),
 (cisco express, 'ORG', 383),
 (v6, 'PRODUCT', 386),
 (cisco waas express, 'ORG', 383),
 (cisco, 'GPE', 384),
 (january 2013 december 2013, 'DATE', 391),
 (december 2013, 'DATE', 391),
 (21, 'CARDINAL', 397),
 (200, 'CARDINAL', 397),
 (535, 'CARDINAL', 397),
 (waas express, 'ORG', 383),
 (cisco waas, 'FAC', 9191306739292312949),
 (waas express, 'ORG', 383),
 (wan, 'PERSON', 380),
 (

In [ ]:
displacy.render(nytimes, style = "ent",jupyter = True)

### TRAINING NER SPACY MODEL

In [ ]:
data

In [15]:
#!pip install spacy==2.1.4

     |████████████████████████████████| 29.8 MB 1.3 MB/s 
     |████████████████████████████████| 2.1 MB 34.1 MB/s 
     |████████████████████████████████| 54 kB 3.8 MB/s 
     |████████████████████████████████| 82 kB 505 kB/s 
     |████████████████████████████████| 3.2 MB 49.9 MB/s 
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.6
    Uninstalling preshed-3.0.6:
      Successfully uninstalled preshed-3.0.6
  Attempting uninstall: plac
    Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Attempting uninstall: blis
    Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.3.3
    Uninstalling jsonschema-4.3.3:
  

In [30]:
def clean_entities(training_data):
    
    clean_data = []
    for text, annotation in training_data:
        
        entities = annotation.get('entities')
        entities_copy = entities.copy()
        
        # append entity only if it is longer than its overlapping entity
        i = 0
        for entity in entities_copy:
            j = 0
            for overlapping_entity in entities_copy:
                # Skip self
                if i != j:
                    e_start, e_end, oe_start, oe_end = entity[0], entity[1], overlapping_entity[0], overlapping_entity[1]
                    # Delete any entity that overlaps, keep if longer
                    if ((e_start >= oe_start and e_start <= oe_end) \
                    or (e_end <= oe_end and e_end >= oe_start)) \
                    and ((e_end - e_start) <= (oe_end - oe_start)):
                        entities.remove(entity)
                j += 1
            i += 1
        clean_data.append((text, {'entities': entities}))
                
    return clean_data

data = clean_entities(data)

In [31]:
import random
import math

def train_test_split(data, test_size, random_state):

    random.Random(random_state).shuffle(data)
    test_idx = len(data) - math.floor(test_size * len(data))
    train_set = data[0: test_idx]
    test_set = data[test_idx: ]

    return train_set, test_set

In [18]:
train_data, test_data = train_test_split(data, test_size = 0.1, random_state = 42)

In [32]:
import spacy

def train_spacy():
    
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:

        parser = nlp.create_pipe('parser')
        nlp.add_pipe(parser, last = True)

        tagger = nlp.create_pipe('tagger')
        nlp.add_pipe(tagger, last = True)

        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
        

        
        
    # add labels
    for _, annotations in train_data:
         for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(30):
            print("Starting iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            for text, annotations in train_data:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

In [ ]:
nlp = train_spacy()

Starting iteration 0
{'ner': 23559.368888100926}
Starting iteration 1
{'ner': 17619.42046299812}
Starting iteration 2
{'ner': 13265.571325725681}
Starting iteration 3
{'ner': 12931.606015396237}
Starting iteration 4
{'ner': 12582.107166120633}
Starting iteration 5
{'ner': 10895.67369372571}
Starting iteration 6
{'ner': 10526.100830206307}
Starting iteration 7
{'ner': 10503.882761465913}
Starting iteration 8
{'ner': 9736.890069631025}
Starting iteration 9
{'ner': 9229.516294115405}
Starting iteration 10
{'ner': 8133.902628460886}
Starting iteration 11
{'ner': 9794.117359284279}
Starting iteration 12
{'ner': 8530.108965252575}
Starting iteration 13
{'ner': 8617.582176139053}
Starting iteration 14
{'ner': 8182.678104281609}
Starting iteration 15
{'ner': 9303.302175488503}
Starting iteration 16
{'ner': 7740.57787644904}
Starting iteration 17
{'ner': 6936.620962986367}
Starting iteration 18
{'ner': 6972.130463028991}
Starting iteration 19
{'ner': 6581.586865959505}
Starting iteration 20
{'n

In [21]:
from spacy.gold import GoldParse
from itertools import groupby

def doc_to_bilou(nlp, text):
    
    doc = nlp(text)
    tokens = [(tok.text, tok.idx, tok.ent_type_) for tok in doc]
    entities = []
    for entity, group in groupby(tokens, key=lambda t: t[-1]):
        if not entity:
            continue
        group = list(group)
        _, start, _ = group[0]
        word, last, _ = group[-1]
        end = last + len(word)
        
        entities.append((
                start,
                end,
                entity
            ))

    gold = GoldParse(nlp(text), entities = entities)
    pred_ents = gold.ner
    
    return pred_ents

y_test = []
y_pred = []

for text, annots in test_data:
    
    gold = GoldParse(nlp.make_doc(text), entities = annots.get("entities"))
    ents = gold.ner
    pred_ents = doc_to_bilou(nlp, text)
    
    y_test.append(ents)
    y_pred.append(pred_ents)
    
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from itertools import chain

def ner_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_)
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset
    ), accuracy_score(y_true_combined, y_pred_combined)
    
report, accuracy = ner_report(y_test, y_pred)
print(report)

                       precision    recall  f1-score   support

                    -       0.00      0.00      0.00       142
       B-College Name       0.69      0.91      0.78        32
       I-College Name       0.51      0.94      0.66        63
       L-College Name       0.64      0.84      0.73        32
       U-College Name       0.00      0.00      0.00         1
B-Companies worked at       0.85      0.73      0.79        30
I-Companies worked at       0.00      0.00      0.00         4
L-Companies worked at       0.81      0.70      0.75        30
U-Companies worked at       0.50      0.34      0.41        41
             B-Degree       0.91      0.83      0.87        24
             I-Degree       0.90      0.92      0.91        66
             L-Degree       0.91      0.83      0.87        24
             U-Degree       0.40      0.67      0.50         3
        B-Designation       0.66      0.81      0.72        47
        I-Designation       0.81      0.53      0.64  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
print(accuracy)

0.8669971374712002


### Testing data

In [23]:
!pip install PyMuPDF

     |████████████████████████████████| 8.8 MB 23.2 MB/s 


In [25]:
import sys, fitz
fname = '/content/Resume_Raghav_2.pdf'
doc = fitz.open(fname)
doc

Document('/content/Resume_Raghav_2.pdf')

In [26]:
text = ''
for page in doc:
    text = text + str(page.get_text())
tx = " ".join(text.split('\n'))
print(tx)

Raghav Rastogi Availability May 2022-Dec 2022 Cherokee Street, Boston • 6177496312 • raghavrastogi75@gmail.com  • https://www.linkedin.com/in/raghav-rastogi-07/ • https://raghavrastogi75.wixsite.com/website Master of Science in Data Science Northeastern University, Khoury College of Computer Sciences, Boston, MA Expected May 2023  GPA : 4.0  Coursework : Algorithms, Database Management, NLP, Supervised Machine Learning  Project: Project: Forecast depression, finding key attributes in survey data and predicting depression based on specific demographic information with an error of around 10% Education B.Tech : Electrical and Electronics Engineering Vellore Institute of Technology, Bangalore, India May 2018  GPA: 8.82/10  Relevant Coursework : Introduction to programming, Data Structures, Probability and Statistics, Multivariable calculus, Linear Algebra Work Experience Associate Solution Advisor July 2020 - August 2021 Deloitte USI, Bangalore, India  Assembled and monitored data re

In [28]:
doc = nlp(tx)
print(doc)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

Raghav Rastogi Availability May 2022-Dec 2022 Cherokee Street, Boston • 6177496312 • raghavrastogi75@gmail.com  • https://www.linkedin.com/in/raghav-rastogi-07/ • https://raghavrastogi75.wixsite.com/website Master of Science in Data Science Northeastern University, Khoury College of Computer Sciences, Boston, MA Expected May 2023  GPA : 4.0  Coursework : Algorithms, Database Management, NLP, Supervised Machine Learning  Project: Project: Forecast depression, finding key attributes in survey data and predicting depression based on specific demographic information with an error of around 10% Education B.Tech : Electrical and Electronics Engineering Vellore Institute of Technology, Bangalore, India May 2018  GPA: 8.82/10  Relevant Coursework : Introduction to programming, Data Structures, Probability and Statistics, Multivariable calculus, Linear Algebra Work Experience Associate Solution Advisor July 2020 - August 2021 Deloitte USI, Bangalore, India  Assembled and monitored data re

### CUSTOM NER MODEL

In [ ]:
nlp.pipe_names

['ner']